In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [9]:
#apariciones primeros 3 dias
appearances_train = pd.read_csv("appearances_first_3_days.csv", header=None, names=["device_id", "date", "seconds_until_next_appearence"])

In [10]:
#apariciones segundos 3 dias
appearances_test =  pd.read_csv("appearances_second_3_days.csv", header=None, names=["device_id", "date", "seconds_until_next_appearence"])

In [11]:
#convertions_train = pd.read_csv("auctions_deltas_to_convertion_train.csv",header=None, names=["device_id", "date", "seconds_until_convertion"])

In [12]:
#convertions_test = pd.read_csv("auctions_deltas_to_convertion_test.csv",header=None, names=["device_id", "date", "seconds_until_convertion"])

In [13]:
#ids a predecir en ambos problemas
ids_to_predict = pd.read_csv("target_competencia_ids.csv")

In [14]:
ids_to_predict = ids_to_predict[["ref_hash"]]

In [15]:
appearances_train["date"] = pd.to_datetime(appearances_train["date"])

In [16]:
appearances_test["date"] = pd.to_datetime(appearances_test["date"])

In [17]:
#convertions_test["date"] = pd.to_datetime(convertions_test["date"])

In [18]:
#convertions_train["date"] = pd.to_datetime(convertions_train["date"])

In [19]:
events = pd.read_csv("events.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
#appearances_test["is_weekend"] = appearances_test.apply(lambda row: row["date"].weekday()>4 , axis=1)

In [21]:
#convertions_test["is_weekend"] = convertions_test.apply(lambda row: row["date"].weekday()>4 , axis=1)

In [22]:
#convertions_train["is_weekend"] = convertions_train.apply(lambda row: row["date"].weekday()>4 , axis=1)

In [23]:
ids_to_predict_numbers = ids_to_predict.copy()

In [24]:
#paso refhash a int sacandole los _sc o _st
ids_to_predict_numbers["ref_hash"] = ids_to_predict_numbers["ref_hash"].apply(lambda string: int(string.split("_")[0]))

In [25]:
#borro duplicados
ids_to_predict_numbers = ids_to_predict_numbers.drop_duplicates()

In [26]:
#por enunciado la fecha de los ids a predecir tiene que ser esta LA MISMA PARA TODOS
#o sea hay que predecir los segundos a partir de esa fecha
ids_to_predict_numbers["date"] = datetime(2019, 5, 27, 0,0,0)

In [27]:
ids_to_predict_numbers.head()

,ref_hash,date
0,1000169251625791246,2019-05-27
2,1000395625957344683,2019-05-27
4,1003027494996471685,2019-05-27
6,1006670001679961544,2019-05-27
8,1007573308966476713,2019-05-27


In [28]:
#la copio en 2 dfs distintos para usar, uno para predecir el tiempo en reaparecer en auction
#y otro para predecir el tiempo en convertir/instalar

In [29]:
ids_to_predict_numbers_appearances = ids_to_predict_numbers.copy()

In [30]:
ids_to_predict_numbers_convertion = ids_to_predict_numbers.copy()

In [31]:
appearances_train.head()

,device_id,date,seconds_until_next_appearence
0,4172466725848941608,2019-04-18 06:27:50.996158,848.908411
1,4172466725848941608,2019-04-18 06:41:59.904569,28513.038578
2,4172466725848941608,2019-04-18 14:37:12.943147,199.094384
3,4172466725848941608,2019-04-18 14:40:32.037531,185.775504
4,4172466725848941608,2019-04-18 14:43:37.813035,255.313251


In [32]:
ids_to_predict_numbers_appearances.head()

,ref_hash,date
0,1000169251625791246,2019-05-27
2,1000395625957344683,2019-05-27
4,1003027494996471685,2019-05-27
6,1006670001679961544,2019-05-27
8,1007573308966476713,2019-05-27


In [33]:
#renombro columna para poder mergear por id
ids_to_predict_numbers_appearances = ids_to_predict_numbers_appearances.rename(columns={"ref_hash": "device_id"})

In [34]:
ids_to_predict_numbers_appearances.head()

,device_id,date
0,1000169251625791246,2019-05-27
2,1000395625957344683,2019-05-27
4,1003027494996471685,2019-05-27
6,1006670001679961544,2019-05-27
8,1007573308966476713,2019-05-27


In [35]:
#les agrego los features

In [36]:
#appearances_train["is_weekend"] = appearances_train["data"].apply(lambda date: date.weekday()>4)

In [37]:
#ids_to_predict_numbers_appearances["is_weekend"] = ids_to_predict_numbers_appearances["date"].apply(lambda date: date["date"].weekday()>4)

In [38]:
#voy a agregar feature de APP mas usada para cada ID
most_used_app = events.groupby("ref_hash")["application_id"].agg(lambda x: x.mode()).rename_axis("device_id").to_frame().reset_index()


In [39]:
#arreglo que algunos casos la app id mas usada es una lista. porque la funcion .mode() devolvio mas de un elemento
most_used_app["application_id"] = most_used_app["application_id"].apply(lambda x: x[0] if type(x) == np.ndarray else x)

In [40]:
#SE LO AGREGO A LOS 3 DFS , cosa que voy a hacer con todos los nuevos features
#porque train (primeros 3 dias) es para entrenar, test (segundos 3 dias) es para testear, y competition es
#para submitear 

In [41]:
df_train = appearances_train.merge(most_used_app, on="device_id", how="left")

In [42]:
df_test = appearances_test.merge(most_used_app, on="device_id", how="left")
df_competition = ids_to_predict_numbers_appearances.merge(most_used_app, on="device_id", how="left")

In [43]:
#renombro para dejar mas claro que el nombre del nuevo feature
df_train = df_train.rename({"application_id": "most_used_app"}, axis="columns")
df_test = df_test.rename({"application_id": "most_used_app"}, axis="columns")
df_competition = df_competition.rename({"application_id": "most_used_app"}, axis="columns")

In [44]:
#reemplazo los nan de most used app por un nuemero no existente
df_train["most_used_app"] = df_train["most_used_app"].fillna(-1)
df_test["most_used_app"] = df_test["most_used_app"].fillna(-1)
df_competition["most_used_app"] = df_competition["most_used_app"].fillna(-1)

In [45]:
def is_android_or_macos(row):
    if row["user_agent"] is np.nan:
        return None
    agent = row["user_agent"].lower()
    if "android" in agent:
        return "android"
    elif "darwin" in agent:
        return "mac os"
    else:
        return "other"

In [46]:
#AHORA USO installs para crear feature de sistema operativo

In [47]:
installs = pd.read_csv("installs.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [48]:
installs["os"] = installs.apply(is_android_or_macos, axis=1)

In [49]:
os = installs[["ref_hash", "os"]].dropna()

In [50]:
#saco ids duplicados
os = os.drop_duplicates(subset="ref_hash", keep='first').rename({"ref_hash": "device_id"},axis="columns")

In [51]:
#lo agrego a DF, siempre left join
df_train = df_train.merge(os,how="left", on="device_id")
df_test = df_test.merge(os,how="left", on="device_id")
df_competition = df_competition.merge(os,how="left", on="device_id")

In [52]:
df_train["os"] = df_train["os"].fillna("other")
df_test["os"] = df_test["os"].fillna("other")
df_competition["os"] = df_competition["os"].fillna("other")

In [53]:
#PRUEBO OTRA VEZ con random forest
#antes tengo que convertir a numeros los campos que son string (en este caso solo OS)
#USO LABEL ENCODER
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [54]:
#encodeo columna os porque no puede ser string. tiene que ser numero
le.fit(df_train["os"])
df_train["os"] = le.transform(df_train["os"])
df_test["os"] = le.transform(df_test["os"])
df_competition["os"] = le.transform(df_competition["os"])

In [55]:
#ahora en vez de string tengo sus codigos
df_train["os"].head()

0    2
1    2
2    2
3    2
4    2
Name: os, dtype: int32

In [56]:
#Agrego features de cantidades de installs, auctions y events

In [6]:
#Se agregan los csv de features
df_eventos_feature = pd.read_csv("df_eventos_feature.csv")
auctions_installs_cantidades = pd.read_csv("auctions_installs_cantidades.csv")
features_cantidades = auctions_installs_cantidades.merge(df_eventos_feature, how = "outer", on = "device_id")

In [57]:
#lo agrego a DF, siempre left join
df_train = df_train.merge(features_cantidades,how="left", on="device_id")
df_test = df_test.merge(features_cantidades,how="left", on="device_id")
df_competition = df_competition.merge(features_cantidades,how="left", on="device_id")


In [59]:
df_train.fillna(0, inplace = True )
df_test.fillna(0, inplace = True )
df_competition.fillna(0, inplace = True)

In [60]:
df_train.columns

Index(['device_id', 'date', 'seconds_until_next_appearence', 'most_used_app',
       'os', 'Unnamed: 0_x', '1_Periodo_Installs', '2_Periodo_Installs',
       '3_Periodo_Installs', 'Cantidad_total_de_install', '1_Periodo_Auctions',
       '2_Periodo_Auctions', '3_Periodo_Auctions', 'Cantidad_total_de_auction',
       'Unnamed: 0_y', 'mejor_evento', 'cant_mejor_evento',
       'cant_total_de_eventos', 'Primer_Periodo', 'Segundo_Periodo',
       'Tercer_Periodo'],
      dtype='object')

In [61]:
df_train["mejor_evento"] = df_train["mejor_evento"].astype('category')

### Aca veo con que features me quedo

In [107]:
features = ["device_id",
       'most_used_app', 'os','cant_total_de_eventos', 'mejor_evento', 'Primer_Periodo', 'Segundo_Periodo',
       'Tercer_Periodo']

In [72]:

#CREO X_TRAIN Y Y_TRAIN con DEVICE ID
X_train = df_train[features]
y_train = df_train['seconds_until_next_appearence']
X_test = df_test[features]
y_test = df_test['seconds_until_next_appearence']
X_competition = df_competition[features]

#### Random Forest

In [73]:

from sklearn.ensemble import RandomForestRegressor

In [74]:
regr = RandomForestRegressor()
#entreno el modeo
regr.fit(X_train, y_train)  
#hago prediccion de X_test
y_pred = regr.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [75]:
from sklearn.metrics import mean_squared_error


In [76]:
#miro error X_test

print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))

Mean squared error: 255429928.01


In [78]:
#ahora predigo los ids que pide el TP
y_pred_competition = regr.predict(X_competition)

In [79]:
prediction_appearances = X_competition[["device_id"]]
prediction_appearances["seconds_until_next_appearance"] = y_pred_competition

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [80]:
prediction_appearances.head()

,device_id,seconds_until_next_appearance
0,1000169251625791246,20000.510115
1,1000395625957344683,5106.490629
2,1003027494996471685,28117.358272
3,1006670001679961544,13677.787211
4,1007573308966476713,46247.165668


In [81]:
#ahora hago la prediccion para tiempo en instalar

In [82]:
ids_to_predict_numbers_convertion = ids_to_predict_numbers_convertion.rename(columns={"ref_hash": "device_id"})

In [83]:
ids_to_predict_numbers_convertion.head()

,device_id,date
0,1000169251625791246,2019-05-27
2,1000395625957344683,2019-05-27
4,1003027494996471685,2019-05-27
6,1006670001679961544,2019-05-27
8,1007573308966476713,2019-05-27


In [84]:
#conversiones de los primeros 3 dias. uso para entrenar
convertions_train = pd.read_csv("auctions_deltas_to_convertion_train.csv",header=None, names=["device_id", "date", "seconds_until_convertion"])

In [86]:
#conversiones de los segunods 3 dias . uso para testear performance
convertions_test = pd.read_csv("auctions_deltas_to_convertion_test.csv",header=None, names=["device_id", "date", "seconds_until_convertion"])

In [87]:
convertions_train.head()

,device_id,date,seconds_until_convertion
0,8323952185725734415,2019-04-20 05:35:45.651502,259200.0
1,8323952185725734415,2019-04-20 05:40:46.744391,259200.0
2,8323952185725734415,2019-04-20 05:48:22.167361,259200.0
3,8323952185725734415,2019-04-20 05:51:43.749231,259200.0
4,8323952185725734415,2019-04-20 05:51:52.533312,259200.0


In [88]:
#agrego mismos features

In [89]:
df_train = convertions_train.merge(most_used_app, on="device_id", how="left")

In [90]:
df_test = convertions_test.merge(most_used_app, on="device_id", how="left")
df_competition = ids_to_predict_numbers_convertion.merge(most_used_app, on="device_id", how="left")

In [91]:
#renombro para dejar mas claro que el nombre del nuevo feature
df_train = df_train.rename({"application_id": "most_used_app"}, axis="columns")
df_test = df_test.rename({"application_id": "most_used_app"}, axis="columns")
df_competition = df_competition.rename({"application_id": "most_used_app"}, axis="columns")

In [92]:
#reemplazo los nan de most used app por un nuemero no existente
df_train["most_used_app"] = df_train["most_used_app"].fillna(-1)
df_test["most_used_app"] = df_test["most_used_app"].fillna(-1)
df_competition["most_used_app"] = df_competition["most_used_app"].fillna(-1)

In [93]:
#lo agrego a DF, siempre left join
df_train = df_train.merge(os,how="left", on="device_id")
df_test = df_test.merge(os,how="left", on="device_id")
df_competition = df_competition.merge(os,how="left", on="device_id")

In [94]:
df_train["os"] = df_train["os"].fillna("other")
df_test["os"] = df_test["os"].fillna("other")
df_competition["os"] = df_competition["os"].fillna("other")

In [95]:
#PRUEBO OTRA VEZ con random forest
#antes tengo que convertir a numeros los campos que son string (en este caso solo OS)
#USO LABEL ENCODER
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [96]:
#encodeo columna os
le.fit(df_train["os"])
df_train["os"] = le.transform(df_train["os"])
df_test["os"] = le.transform(df_test["os"])
df_competition["os"] = le.transform(df_competition["os"])

In [97]:
#lo agrego a DF, siempre left join
df_train = df_train.merge(features_cantidades,how="left", on="device_id")
df_test = df_test.merge(features_cantidades,how="left", on="device_id")
df_competition = df_competition.merge(features_cantidades,how="left", on="device_id")

In [98]:
df_train.fillna(0, inplace = True )
df_test.fillna(0, inplace = True )
df_competition.fillna(0, inplace = True)

In [99]:
df_train.columns

Index(['device_id', 'date', 'seconds_until_convertion', 'most_used_app', 'os',
       'Unnamed: 0_x', '1_Periodo_Installs', '2_Periodo_Installs',
       '3_Periodo_Installs', 'Cantidad_total_de_install', '1_Periodo_Auctions',
       '2_Periodo_Auctions', '3_Periodo_Auctions', 'Cantidad_total_de_auction',
       'Unnamed: 0_y', 'mejor_evento', 'cant_mejor_evento',
       'cant_total_de_eventos', 'Primer_Periodo', 'Segundo_Periodo',
       'Tercer_Periodo'],
      dtype='object')

#### Random Forest

In [108]:
features = ["device_id",
       'most_used_app', 'os','cant_total_de_eventos', 'Cantidad_total_de_install', 'Cantidad_total_de_auction']

In [104]:

#CREO X_TRAIN Y Y_TRAIN con DEVICE ID
X_train = df_train[features]
y_train = df_train['seconds_until_convertion']
X_test = df_test[features]
y_test = df_test['seconds_until_convertion']
X_competition = df_competition[features]

In [105]:
regr = RandomForestRegressor()
#entreno el modeo
regr.fit(X_train, y_train)  
y_pred = regr.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [106]:
#miro error X_test

print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))

Mean squared error: 5140737155.18


In [74]:
#ahora predigo los ids que pide el TP
y_pred_competition = regr.predict(X_competition)

In [75]:
prediction_convertion = X_competition[["device_id"]]
prediction_convertion["seconds_until_convertion"] = y_pred_competition

/home/martinrosas/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
#miro como me quedaron los dataframes de predicciones
#para submitear tiene que quedar con el formato que piden. o sea los ids de appearances tienen que 
#terminar en _st y los ids de convertions tienen que terminar en _sc

In [76]:
prediction_appearances.head()

,device_id,seconds_until_next_appearance
0,1000169251625791246,14835.445853
1,1000395625957344683,2589.705042
2,1003027494996471685,25653.195744
3,1006670001679961544,5126.502494
4,1007573308966476713,49137.153869


In [77]:
prediction_convertion.head()

,device_id,seconds_until_convertion
0,1000169251625791246,259200.000000
1,1000395625957344683,91770.887781
2,1003027494996471685,259200.000000
3,1006670001679961544,240456.613382
4,1007573308966476713,259200.000000


In [156]:
#paso al formato de submit

In [78]:
prediction_appearances = prediction_appearances.rename(columns={"device_id": "ref_hash", "seconds_until_next_appearance": "obj"})

In [79]:
prediction_convertion = prediction_convertion.rename(columns={"device_id": "ref_hash", "seconds_until_convertion": "obj"})

In [80]:
prediction_appearances["ref_hash"] = prediction_appearances["ref_hash"].apply(lambda ref: str(ref)+"_st")

In [81]:
prediction_convertion["ref_hash"] = prediction_convertion["ref_hash"].apply(lambda ref: str(ref)+"_sc")

In [82]:
prediction_appearances.head()

,ref_hash,obj
0,1000169251625791246_st,14835.445853
1,1000395625957344683_st,2589.705042
2,1003027494996471685_st,25653.195744
3,1006670001679961544_st,5126.502494
4,1007573308966476713_st,49137.153869


In [83]:
prediction_convertion.head()

,ref_hash,obj
0,1000169251625791246_sc,259200.000000
1,1000395625957344683_sc,91770.887781
2,1003027494996471685_sc,259200.000000
3,1006670001679961544_sc,240456.613382
4,1007573308966476713_sc,259200.000000


In [84]:
#concateno las predicciones porque tienen que devolverse todas juntas

In [85]:
full_pred = pd.concat([prediction_appearances, prediction_convertion])

In [86]:
#hago un left join con ids to predict para oredenarlos como pide el enunciado

In [87]:
ids_to_predict.head()

,ref_hash
0,1000169251625791246_sc
1,1000169251625791246_st
2,1000395625957344683_sc
3,1000395625957344683_st
4,1003027494996471685_sc


In [88]:
full_pred = ids_to_predict.merge(full_pred, on="ref_hash", how="left")

In [89]:
#ya esta igualito a como pide el enunciado
full_pred.head()

,ref_hash,obj
0,1000169251625791246_sc,259200.000000
1,1000169251625791246_st,14835.445853
2,1000395625957344683_sc,91770.887781
3,1000395625957344683_st,2589.705042
4,1003027494996471685_sc,259200.000000


In [90]:
#paso float de obj a int
full_pred["obj"] = full_pred["obj"].astype(np.int64)

In [91]:
full_pred.to_csv("first_pred.csv", index=False)

In [92]:
full_pred.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8074 entries, 0 to 8073
Data columns (total 2 columns):
ref_hash    8074 non-null object
obj         8074 non-null int64
dtypes: int64(1), object(1)
memory usage: 189.2+ KB


In [93]:
pd.read_csv("target_competencia_ids.csv").info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8074 entries, 0 to 8073
Data columns (total 2 columns):
ref_hash    8074 non-null object
obj         8074 non-null int64
dtypes: int64(1), object(1)
memory usage: 126.2+ KB
